In [37]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession


keyfile_path = "btcanalysishust-b10a2ef12088.json"

spark = SparkSession.builder \
    .appName("KafkaConsumer") \
  .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.gs.auth.type", "OAuth2") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust") \
    .config("spark.hadoop.fs.gs.input.close.input.streams.after.task.complete", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", keyfile_path) \
    .getOrCreate()



In [ ]:
# Đọc dữ liệu từ Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-prices") \
    .option("startingOffsets", "earliest") \
    .load()

# Kafka data có các cột "key" và "value" dưới dạng binary
df = df.selectExpr("CAST(value AS STRING) as value")

from pyspark.sql.types import StructType, StructField, StringType, FloatType

schema_log = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("binancecoin", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),        
    ]))
])

from pyspark.sql.functions import col, from_json, explode, udf

crypto_parsed_df = df.select(from_json(col("value"), schema_log).alias("data"))


crypto_parsed_df = crypto_parsed_df.select(
    col("data.timestamp").alias("timestamp"),
    col("data.prices.bitcoin").alias("BTC"),
    col("data.prices.ethereum").alias("ETH"),
    col("data.prices.tether").alias("TETHER"),
    col("data.prices.binancecoin").alias("BNB"),
    col("data.prices.usd-coin").alias("USDC"),
    col("data.prices.ripple").alias("XRP"),
    col("data.prices.cardano").alias("ADA"),
    col("data.prices.dogecoin").alias("DOGE"),
    col("data.prices.matic-network").alias("MATIC"),
    col("data.prices.solana").alias("SOLANA")
)


# Hiển thị dữ liệu đọc được ra console (cho thử nghiệm), không cắt ngắn các giá trị
crypto_parsed_df.writeStream.format("console").option("truncate", "false").start().awaitTermination()


In [38]:
df = spark.read.parquet("gs://crypto-historical-data-2/ver2/BTC/2024/01/data.parquet")
df.show()


+----+----------+--------+--------+--------+--------+--------------+----+-----+-----------------+
|BASE|      DATE|    OPEN|    HIGH|     LOW|   CLOSE|        VOLUME|YEAR|MONTH|__index_level_0__|
+----+----------+--------+--------+--------+--------+--------------+----+-----+-----------------+
| BTC|2024-01-31|42942.78|43741.94|42265.77|42548.08|15320.62044085|2024|    1|              299|
| BTC|2024-01-30|43306.18|43872.99| 42680.2|42942.78|16384.61219735|2024|    1|              300|
| BTC|2024-01-29|42034.59|43326.77|41787.35|43306.18|12529.09255982|2024|    1|              301|
| BTC|2024-01-28|42126.55|42839.07|41617.53|42034.59| 4298.44263812|2024|    1|              302|
| BTC|2024-01-27| 41814.8|42199.99|41383.94|42126.55| 3774.45056434|2024|    1|              303|
| BTC|2024-01-26|39941.66|42243.14|39805.95| 41814.8|19359.37921096|2024|    1|              304|
| BTC|2024-01-25|40077.78|40282.87| 39503.0|39941.66| 12978.7331205|2024|    1|              305|
| BTC|2024-01-24| 39

In [67]:
from google.cloud import storage
from datetime import datetime
from pyspark.sql.functions import to_date
from dateutil.relativedelta import relativedelta
def get_last_saved_date(crypto_id, storage_path : str):
    """
    Kiểm tra ngày cuối cùng đã được lưu trữ trong GCS hoặc HDFS.
    """
    storage_client = storage.Client.from_service_account_json("btcanalysishust-b10a2ef12088.json")
    blobs = storage_client.list_blobs(storage_path, prefix=f"ver2/{crypto_id}/")
    last_saved_date = None
    for blob in blobs:
        # Trích xuất ngày từ tên thư mục
        path_parts = blob.name.split('/')

        if len(path_parts) > 2: 
            year, month = path_parts[2], path_parts[3]
            date = datetime.strptime(f"{year}-{month.zfill(2)}","%Y-%m")
            if last_saved_date is None or date > last_saved_date:
                last_saved_date = date

    return last_saved_date.strftime("%Y-%m")

def get_gcs_price(crypto_id : str, start_date : str , end_date : str):
    """
    Lấy dữ liệu giá từ GCS trong khoảng thời gian chỉ định và hợp nhất thành một bảng Spark.

    Args:
        crypto_id (str): Tên tài sản (crypto, cổ phiếu, v.v.).
        start_date (str): Ngày bắt đầu (định dạng 'YYYY-MM').
        end_date (str): Ngày kết thúc (định dạng 'YYYY-MM').

    Returns:
        DataFrame: Bảng Spark chứa tất cả dữ liệu giá hợp nhất.
    """
    # Chuyển đổi chuỗi ngày thành đối tượng datetime
    start = datetime.strptime(start_date, "%Y-%m")
    end = datetime.strptime(end_date, "%Y-%m")
    
    # Kiểm tra ngày bắt đầu phải nhỏ hơn hoặc bằng ngày kết thúc
    if start > end:
        raise ValueError("start_date phải nhỏ hơn hoặc bằng end_date")
    
    # Khởi tạo danh sách kết quả
    all_prices = []
    
    # Lặp qua từng tháng
    current = start.replace(day=1)  # Đặt ngày thành ngày đầu tiên của tháng
    while current <= end:
        curr_price_dir=f"gs://crypto-historical-data-2/ver2/{crypto_id}/{current.year}/{current.month:02}/data.parquet"
        all_prices.append(spark.read.parquet(curr_price_dir))
        if current.month == 12:  # Nếu là tháng 12, chuyển sang tháng 1 năm sau
            current = current.replace(year=current.year + 1, month=1)
        else:
            current = current.replace(month=current.month + 1)
    if all_prices:
        merged_data = all_prices[0]
        for df in all_prices[1:]:
            merged_data = merged_data.union(df)
        merged_data = merged_data.withColumn('DATE_TRANSFORMED', to_date('DATE', 'yyyy-MM-dd'))

        # # Filter using the transformed date column
        # filtered_data = merged_data.filter(
        #     (merged_data['DATE_TRANSFORMED'] >= start_date) & 
        #     (merged_data['DATE_TRANSFORMED'] <= end_date)
        # )

        # Sort by the transformed date column
        sorted_data = merged_data.orderBy('DATE_TRANSFORMED', ascending= False)

        # Sort the DataFrame by the date column
        return sorted_data
    else:
        raise ValueError("Không tìm thấy dữ liệu trong khoảng thời gian được chỉ định.")

df=get_gcs_price('BTC', '2023-12','2024-1')

df.show(n=1000)

+----+----------+--------+--------+--------+--------+--------------+----+-----+-----------------+----------------+
|BASE|      DATE|    OPEN|    HIGH|     LOW|   CLOSE|        VOLUME|YEAR|MONTH|__index_level_0__|DATE_TRANSFORMED|
+----+----------+--------+--------+--------+--------+--------------+----+-----+-----------------+----------------+
| BTC|2024-01-31|42942.78|43741.94|42265.77|42548.08|15320.62044085|2024|    1|              299|      2024-01-31|
| BTC|2024-01-30|43306.18|43872.99| 42680.2|42942.78|16384.61219735|2024|    1|              300|      2024-01-30|
| BTC|2024-01-29|42034.59|43326.77|41787.35|43306.18|12529.09255982|2024|    1|              301|      2024-01-29|
| BTC|2024-01-28|42126.55|42839.07|41617.53|42034.59| 4298.44263812|2024|    1|              302|      2024-01-28|
| BTC|2024-01-27| 41814.8|42199.99|41383.94|42126.55| 3774.45056434|2024|    1|              303|      2024-01-27|
| BTC|2024-01-26|39941.66|42243.14|39805.95| 41814.8|19359.37921096|2024|    1| 

In [ ]:
class STOCK:
    def __init__(self, period_k=9, period_d=6, storage_path='crypto-historical-data-2' ):
        self.k=period_k
        self.d=period_d
        self.storage_path=storage_path
    def calculate_stock(self, crypto_id):
        lastest_month=get_last_saved_date(crypto_id,self.storage_path)
        comeback_month=(self.k+self.d)//28+1
        start_month=datetime.strptime(lastest_month,'%Y-%m')-relativedelta(months=comeback_month)
        historical_data=get_gcs_price(crypto_id,start_month.strftime('%Y-%m'), lastest_month)
        # gia tri gia su
        current_close=12
        
cal_stock=STOCK(15,15)
cal_stock.calculate_stock('BTC')

2024-09 2024-11


In [74]:
30//28

1

In [66]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Example lastest_month
lastest_month = '2024-11'

# Parse the string into a datetime object
current_date = datetime.strptime(lastest_month, '%Y-%m')

# Subtract 2 months using relativedelta
two_months_back = current_date - relativedelta(months=2)

# Format the result back to 'YYYY-MM' string
two_months_back_str = two_months_back.strftime('%Y-%m')

print(two_months_back_str)  # Output: 2024-09


2024-09
